In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
from fucciphase.phase import (
    estimate_percentage_by_subsequence_alignment,
)
from scipy import interpolate

In [ ]:
if not os.path.isdir("figures"):
    os.mkdir("figures")

## Read the reference curve

In [ ]:
reference_file = "example_data/hacat_fucciphase_reference.csv"
reference_df = pd.read_csv(reference_file)
# add a fake TRACK_ID
reference_df["TRACK_ID"] = 1
timestep = reference_df["time"][1] - reference_df["time"][0]
cyan_channel = "cyan"
magenta_channel = "magenta"

# Create a test curve

* take selected datapoints
* scale the intensity or introduce an offset

In [ ]:
start_idx = 30
end_idx = 70
df = reference_df.iloc[start_idx:end_idx]
df["cyan"] = df.loc[:, "cyan"] + 0.5
df["magenta"] = df.loc[:, "magenta"] * 2

In [ ]:
estimate_percentage_by_subsequence_alignment(
        df,
        dt=timestep,
        channels=[cyan_channel, magenta_channel],
        reference_data=reference_df,
        track_id_name="TRACK_ID")


In [ ]:
plt.plot(df["percentage"], df["CELL_CYCLE_PERC_DTW"])
plt.xlabel("Expected percentage")
plt.ylabel("Estimated percentage")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
reference_df["percentage"]
ax1.plot(df["CELL_CYCLE_PERC_DTW"], df["cyan"], label="Query")
ax1.plot(reference_df["percentage"], reference_df["cyan"], label="Reference")
f_cyan = interpolate.interp1d(reference_df["percentage"], reference_df["cyan"])
ax1.plot(df["CELL_CYCLE_PERC_DTW"], f_cyan(df["CELL_CYCLE_PERC_DTW"]),
         lw=6,
         alpha=0.5,
         color="red",
         label="Match",
)
ax1.plot(reference_df["percentage"].iloc[start_idx:end_idx],
         reference_df["cyan"].iloc[start_idx:end_idx],
         label="Ground truth",
         lw=3)

ax2.plot(df["CELL_CYCLE_PERC_DTW"], df["magenta"], label="Query")
ax2.plot(reference_df["percentage"], reference_df["magenta"], label="Reference")
f_magenta = interpolate.interp1d(reference_df["percentage"], reference_df["magenta"])
ax2.plot(df["CELL_CYCLE_PERC_DTW"], f_magenta(df["CELL_CYCLE_PERC_DTW"]),
         lw=6,
         alpha=0.5,
         color="red",
)
ax2.plot(reference_df["percentage"].iloc[start_idx:end_idx],
         reference_df["magenta"].iloc[start_idx:end_idx],
         lw=3)

ax1.set_ylabel("Cyan intensity / arb. u.")
ax2.set_ylabel("Magenta intensity / arb. u.")
ax1.set_xlabel("Cell cycle percentage")
ax2.set_xlabel("Cell cycle percentage")
ax1.legend()
plt.tight_layout()
plt.savefig("explain_dtw_query.pdf")
plt.savefig("explain_dtw_query.svg")